In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from tensorflow import set_random_seed
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor
from sklearn.grid_search import GridSearchCV 
from keras import regularizers

from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import StratifiedKFold
np.random.seed(42)


Using TensorFlow backend.
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## 1.读取-划分数据

In [2]:
def importdata():
    path = r"C:\Users\hasee\workspace\workspace\tomo\data2.txt"
    df = pd.read_table(path)
    columns = df.columns
    #print(columns)

    dfdata = df[['latgpsutm', 'lnggpsutm', 'lnglacutm', 'latlacutm']]
    #print(len(dfdata))
    dfdata = dfdata.drop_duplicates()#删去重复项
    #print(len(dfdata))
    return dfdata

df = importdata()

随机70%训练集切分-方法1：

In [3]:
df1 = shuffle(df)
df1 = df.set_index([list(range(len(df1)))])
index = int(len(df1)*0.7)
df1_train=df1[:index]
df1_test=df1[index:]

随机70%训练集切分-方法2：

In [4]:
df_train=df.sample(frac=0.7)
df_train=df_train.reset_index(drop=True)

In [5]:
df_train.isnull().values.any()

False

In [6]:
list(df_train)

['latgpsutm', 'lnggpsutm', 'lnglacutm', 'latlacutm']

In [7]:
X_train=df_train[['lnglacutm', 'latlacutm']].as_matrix()
y_train=df_train[['latgpsutm', 'lnggpsutm']].as_matrix()

In [8]:
X_train

array([[  353272.057 ,  3451367.8369],
       [  357161.5617,  3462181.514 ],
       [  352815.7743,  3456243.1191],
       ..., 
       [  356961.2242,  3462713.0734],
       [  351431.9022,  3452487.3429],
       [  353368.9606,  3451370.178 ]])

# Build the model

In [15]:
def NN_model(optimizer='rmsprop',hidden_units=100,dropout_rate=0.3,L2_rate=.001):
    model=Sequential()
    model.add(Dense(hidden_units,activation='tanh',input_dim=2,kernel_regularizer=regularizers.l2(L2_rate)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(2,activation='tanh',kernel_regularizer=regularizers.l2(L2_rate)))
    
    model.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['accuracy'])
    return model

In [16]:
nn_model=NN_model()
#nn_reg=KerasRegressor(build_fn=nn_model,nb_epoch=40,batch_size=50,verbose=-1)


In [17]:
nn_model.fit(X_train,y_train,epochs=30,batch_size=50,verbose=-1)#callbacks='early_stop')

Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [19]:
nn_model.evaluate(X_train,y_train)

1117/1117 [==============================] - ETA: 0s


[6043603479758.2666, 1.0]

In [35]:
test_X=df1_test[['lnglacutm', 'latlacutm']].iloc[0:2].as_matrix()
test_y=df1_test[['latgpsutm', 'lnggpsutm']].iloc[0:2].as_matrix()

In [37]:
nn_model.predict(test_X)

array([[ 1.,  1.],
       [ 1.,  1.]], dtype=float32)